## Scrape with scrapy

In [3]:
import requests
from itertools import chain
import numpy as np
import pandas as pd
import time


from scrapy.selector import Selector
from scrapy.http import HtmlResponse

## www.mycareersfuture.sg

#### Test waters

In [11]:
# Target web page:
url = "https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=0"
# Establishing the connection to the web page:
response = requests.get(url)

# You can use status codes to understand how the target server responds to your request.
# Ex., 200 = OK, 400 = Bad Request, 403 = Forbidden, 404 = Not Found.
print('Status Code: ',response.status_code)

# Pull the HTML string out of requests and convert it to a Python string.
html = response.text

# The first 500 characters of the content.
print("\nFirst part of HTML document fetched as string:\n")
print(html[:500])
print(html)

Status Code:  200

First part of HTML document fetched as string:

<!DOCTYPE html>
<html lang="en">
<head>
  <title>
    MyCareersFuture 
  </title>
  <meta property="og:title" content="MyCareersFuture " />
  <meta property="og:description" content="The newest government initiative to help Singaporeans with a smarter way to find jobs. You can now search for jobs based on your skills, discover suitable jobs you might not have considered before, or find jobs with government support." />
  <meta property="og:url" content="https://www.mycareersfuture.sg" />
  <meta
<!DOCTYPE html>
<html lang="en">
<head>
  <title>
    MyCareersFuture 
  </title>
  <meta property="og:title" content="MyCareersFuture " />
  <meta property="og:description" content="The newest government initiative to help Singaporeans with a smarter way to find jobs. You can now search for jobs based on your skills, discover suitable jobs you might not have considered before, or find jobs with government support." />
  <meta p

#### Actual scraping

In [79]:
def parse_mycareersfuture_job_details(url):
    
    response  =  requests.get(url)
    print('Status Code: ',response.status_code)

    root_sel = Selector(text=response.text)        
    
    job_title = root_sel.xpath("//h3[@class='icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title']/text()").extract()
    company   =  root_sel.xpath("//div[@class='icl-u-lg-mr--sm icl-u-xs-mr--xs'][1]/text()").extract()
    location    =  root_sel.xpath("//div[@class='jobsearch-InlineCompanyRating icl-u-xs-mt--xs  icl-u-xs-mb--md']/div[3]/text()").extract()
    salary = null
    
    print('Job_title: ' + job_title[0])
    print('Company: ' + company[0])
    if len(location) > 0:
        print('Location: ' + location[0])
    else:
        location = 'Singapore'
    
    print('Parsing job description now...')
    job_description = ''
    job_description_sel = root_sel.xpath("//div[@class='jobsearch-JobComponent icl-u-xs-mt--sm']/div[@class='jobsearch-JobComponent-description icl-u-xs-mt--md']")
    for i, sel in enumerate(job_description_sel):
        print(i)
        print(sel.extract())

    

    
    df = pd.DataFrame.from_dict(scraped)
    
    return df

# this has salary
#parse_indeed_job_details(url='https://www.indeed.com.sg/cmp/Rakuten-Asia-Pte-Ltd/jobs/Data-Technologist-5d43d7d3d072e772?q=data+related&vjs=3')

# this has no salary                                  
parse_indeed_job_details(url='https://www.indeed.com.sg/viewjob?jk=95dbbcef5755e6bc&tk=1cqqda7827cdk800&from=serp&alid=3&advn=8858013441684577')

SSLError: HTTPSConnectionPool(host='www.indeed.com.sg', port=443): Max retries exceeded with url: /viewjob?jk=95dbbcef5755e6bc&tk=1cqqda7827cdk800&from=serp&alid=3&advn=8858013441684577 (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

In [7]:
a = 1 if 1 > 2 else 11
a

11

In [10]:
def parse_mycareersfuture_search_page(url, data=False):
    
    response  =  requests.get(url)
    print('Status Code: ',response.status_code)

    main_sel = Selector(text=response.text)    

    for job_sel in (main_sel.xpath('//div[@class="card-list"]/div')):
        job_title = job_sel.xpath('.//h1[@name="job_title"]/text()').extract_first()
        job_company = job_sel.xpath('.//p[@name="company"]/text()').extract_first()      

        job_location =  job_sel.xpath('.//p[@name="location"]/text()').extract_first()
        job_senority = job_sel.xpath('.//p[@name="senority"]/text()').extract_first()
        job_category = job_sel.xpath('.//p[@name="category"]/text()').extract_first()
        job_employment_type = job_sel.xpath('.//p[@name="employment_type"]/text()').extract_first()

        # salary
        job_salary_range = job_sel.xpath('.//span[@class="salary_range"]/div/span[@class="dib"]').extract()
        job_salary_range_from = job_salary_range[0], job_salary_range[0]
        job_salary_range_to = job_salary_range[1] if len(job_salary_range) > 1 else job_salary_range[1]
        job_salary_type = job_sel.xpath('.//span[@class="salary_type"]').extract_first() 
        
        job_details_url  =  "https://www.mycareersfuture.sg" % job_details_link
        print("Fetching %s..." % job_details_url)       
        #job_description = parse_indeed_job_details(job_details_url)

        # If there's data, append them. 
        # If not, it's the first iteration
        scraped = dict(
            job_title   =  job_title, 
            job_company    =  job_company,
            job_location = job_location,
            job_senority = job_senority,
            job_category = job_category,
            job_employment_type = job_employment_type,
            job_salary_range_from = job_salary_range_from,
            job_salary_range_to = job_salary_range_to,
            job_salary_type = job_salary_type,
            job_details_url = job_details_url
            #job_description = job_description
        )   

        if type(data) != bool:
             df = pd.DataFrame.from_dict(scraped)
             data = data.append(job_data)
        else:
             data = job_data
                 
    # Find "more" link.
#     more_anchor  =  main_sel.xpath('//span[@type="action"]/text()").extract()
#     if '>' in more_anchor:
#         url_core,current_page_number = url.split('page=')
#         more_url  =  url_core.join('page=%s' % (int(current_page_number)+1))
#         print("Fetching Next Page %s..." % more_url)
#         time.sleep(2)        
#         return parse_mycareersfuture_search_page(more_url, data=data)
#     else:
#         return data.reset_index()

                              
df = parse_mycareersfuture_search_page(url="https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=0")
df

Status Code:  200
